# Import Libraries

In [1]:
import time
from qwikidata.sparql  import return_sparql_query_results
from qwikidata.linked_data_interface import get_entity_dict_from_api
import pandas as pd
import numpy as np
import pickle as pkl
from collections import Counter
import urllib.request
import json
import string
from difflib import SequenceMatcher
from viapy.api import ViafAPI
import re
import math
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
location_to_save = "data/final_files/"
large_location_to_save = "data/final_files/large_files/"

# Let's see.

In [3]:
def remove_redundant_books(author_df):
    author_df.alt_title = author_df.alt_title.apply(lambda x: simplify(x))
    unique_titles = author_df.alt_title.unique()
    author_df_alt = author_df.copy()
    for unique_title in unique_titles:
        first_ISBN = author_df.loc[author_df.alt_title == unique_title].iloc[0].ISBN
        author_df_alt.loc[author_df_alt.alt_title == unique_title, "ISBN_alt"] = first_ISBN
        ISBN_dict = dict(zip(author_df_alt.ISBN, author_df_alt.ISBN_alt))
    return(ISBN_dict)

In [4]:
def simplify(name):
    name = name.replace(" ","").translate(str.maketrans('', '', string.punctuation)).lower()
    return name

In [9]:
entire_books = pd.read_csv(location_to_save+"items_books_with_readable_properties.csv", low_memory=False, index_col=0).fillna("")

In [10]:
entire_books["ISBN_alt"] = ""

In [11]:
unique_authors = entire_books[["QID", "viaf_id"]].drop_duplicates()

In [12]:
len(unique_authors)

82180

In [ ]:
start = time.time()
large_ISBN_dict = {}
for index, row in unique_authors.iterrows():
    viaf_id = row.viaf_id
    QID = row.QID
    author_df = entire_books[(entire_books.QID==QID)&(entire_books.viaf_id==viaf_id)][["ISBN", 
                                                                                       "title",
                                                                                      "alt_title",
                                                                                      "ISBN_alt"]]
    ISBN_dict = remove_redundant_books(author_df)
    large_ISBN_dict.update(ISBN_dict)
    del author_df
    del ISBN_dict
    if index%100==0:
        print("100 is gone!!!!!!!!!", index)
        print(time.time()-start)
        start = time.time()
        with open('ISBN_dict.pkl', 'wb') as handle:
            pkl.dump(large_ISBN_dict, handle)


100 is gone!!!!!!!!! 0
0.040899038314819336
100 is gone!!!!!!!!! 100
27.10940933227539
100 is gone!!!!!!!!! 400
13.479390144348145
100 is gone!!!!!!!!! 500
3.5068862438201904
100 is gone!!!!!!!!! 800
10.988209247589111
100 is gone!!!!!!!!! 1000
7.856187343597412
100 is gone!!!!!!!!! 1200
6.317482948303223
100 is gone!!!!!!!!! 1300
3.197606325149536
100 is gone!!!!!!!!! 1400
3.6008856296539307
100 is gone!!!!!!!!! 1700
9.843211650848389
100 is gone!!!!!!!!! 1800
2.929147720336914
100 is gone!!!!!!!!! 1900
2.9721314907073975
100 is gone!!!!!!!!! 2200
8.788050651550293
100 is gone!!!!!!!!! 2300
2.4214084148406982
100 is gone!!!!!!!!! 2400
2.676874876022339
100 is gone!!!!!!!!! 2500
2.2999141216278076
100 is gone!!!!!!!!! 3400
24.041125774383545
100 is gone!!!!!!!!! 3500
2.1275362968444824
100 is gone!!!!!!!!! 3600
2.9546186923980713
100 is gone!!!!!!!!! 3700
2.1496076583862305
100 is gone!!!!!!!!! 3900
4.425597667694092
100 is gone!!!!!!!!! 4100
4.606888055801392
100 is gone!!!!!!!!! 4300

100 is gone!!!!!!!!! 48700
2.2864527702331543


In [99]:
len(large_ISBN_dict.keys())

18727

In [100]:
len(np.unique(list(large_ISBN_dict.keys())))

18727

In [101]:
len(np.unique(list(large_ISBN_dict.values())))

12167

# Result

In [8]:
with open('ISBN_dict.pkl', 'rb') as handle:
        ISBN_dict = pkl.load(handle)

EOFError: Ran out of input